<a href="https://colab.research.google.com/github/sharmin91/BanglaKeyphraseExtraction/blob/master/Copy_of_SimpleRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sharmin91/BanglaKeyphraseExtraction.git

Cloning into 'BanglaKeyphraseExtraction'...
remote: Enumerating objects: 3508, done.
remote: Counting objects: 100% (3508/3508), done.
remote: Compressing objects: 100% (3488/3488), done.
remote: Total 3508 (delta 40), reused 3441 (delta 13), pack-reused 0
Receiving objects: 100% (3508/3508), 22.63 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [2]:
!mv -v BanglaKeyphraseExtraction/* .

renamed 'BanglaKeyphraseExtraction/AnswerRNN2.py' -> './AnswerRNN2.py'
renamed 'BanglaKeyphraseExtraction/AnswerRNN3.py' -> './AnswerRNN3.py'
renamed 'BanglaKeyphraseExtraction/AnswerRNN.py' -> './AnswerRNN.py'
renamed 'BanglaKeyphraseExtraction/bengali-word-embedding.txt' -> './bengali-word-embedding.txt'
renamed 'BanglaKeyphraseExtraction/data' -> './data'
renamed 'BanglaKeyphraseExtraction/dataset.csv' -> './dataset.csv'
renamed 'BanglaKeyphraseExtraction/DatasetPatternAnalyzer.py' -> './DatasetPatternAnalyzer.py'
renamed 'BanglaKeyphraseExtraction/eval' -> './eval'
renamed 'BanglaKeyphraseExtraction/figures' -> './figures'
renamed 'BanglaKeyphraseExtraction/get_glove.sh' -> './get_glove.sh'
renamed 'BanglaKeyphraseExtraction/LICENSE' -> './LICENSE'
renamed 'BanglaKeyphraseExtraction/MergeRNN2.py' -> './MergeRNN2.py'
renamed 'BanglaKeyphraseExtraction/MergeRNN.py' -> './MergeRNN.py'
renamed 'BanglaKeyphraseExtraction/nlp' -> './nlp'
renamed 'BanglaKeyphraseExtraction/RandomnessDetec

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

path = Path(os.getcwd())
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [6]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-12-30 21:27:47--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-12-30 21:27:47--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-12-30 21:27:48--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [7]:
!unzip glove.6B.zip -d .

Archive:  glove.6B.zip
  inflating: ./glove.6B.50d.txt      
  inflating: ./glove.6B.100d.txt     
  inflating: ./glove.6B.200d.txt     
  inflating: ./glove.6B.300d.txt     


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/fchollet/keras/issues/2280#issuecomment-306959926

import os

# os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(421)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(123451)

import logging

from keras import regularizers
from keras.layers import Bidirectional, Dense, Dropout, Embedding, LSTM, TimeDistributed
from keras.models import Sequential, load_model

from data.datasets import *
from eval import keras_metrics, metrics
from nlp import tokenizer as tk
from utils import info, preprocessing, postprocessing, plots

# LOGGING CONFIGURATION

logging.basicConfig(
    format='%(asctime)s\t%(levelname)s\t%(message)s',
    level=logging.DEBUG)

info.log_versions()

# END LOGGING CONFIGURATION

# GLOBAL VARIABLES

SAVE_MODEL = False
MODEL_PATH = "models/simplernn.h5"
SHOW_PLOTS = False

# END GLOBAL VARIABLES

# Dataset and hyperparameters for each dataset

DATASET = Bangla

if DATASET == Semeval2017:
    tokenizer = tk.tokenizers.nltk
    DATASET_FOLDER = "data/Semeval2017"
    MAX_DOCUMENT_LENGTH = 400
    MAX_VOCABULARY_SIZE = 20000
    EMBEDDINGS_SIZE = 50
    BATCH_SIZE = 32
    EPOCHS = 30
    KP_WEIGHT = 10
    STEM_MODE = metrics.stemMode.both
    STEM_TEST = False
elif DATASET == Bangla:
    tokenizer = tk.tokenizers.nltk
    DATASET_FOLDER = "data/Bangla"
    MAX_DOCUMENT_LENGTH = 550
    MAX_VOCABULARY_SIZE = 20000
    EMBEDDINGS_SIZE = 300
    BATCH_SIZE = 32
    EPOCHS = 28
    KP_WEIGHT = 10
    STEM_MODE = metrics.stemMode.none
    STEM_TEST = False
elif DATASET == Marujo2012:
    tokenizer = tk.tokenizers.nltk
    DATASET_FOLDER = "data/Marujo2012"
    MAX_DOCUMENT_LENGTH = 7000
    MAX_VOCABULARY_SIZE = 20000
    EMBEDDINGS_SIZE = 50
    BATCH_SIZE = 16
    EPOCHS = 10
    KP_WEIGHT = 10
    STEM_MODE = metrics.stemMode.both
    STEM_TEST = False
elif DATASET == Semeval2010:
    tokenizer = tk.tokenizers.nltk
    DATASET_FOLDER = "data/Semeval2010"
    MAX_DOCUMENT_LENGTH = 16600
    MAX_VOCABULARY_SIZE = 50000
    EMBEDDINGS_SIZE = 50
    BATCH_SIZE = 16
    EPOCHS = 40
    KP_WEIGHT = 500
    STEM_MODE = metrics.stemMode.results
    STEM_TEST = True
else:
    raise NotImplementedError("Can't set the hyperparameters: unknown dataset")

# END PARAMETERS

logging.info("Loading dataset...")

data = DATASET(DATASET_FOLDER)

train_doc_str, train_answer_str = data.load_train()
test_doc_str, test_answer_str = data.load_test()
val_doc_str, val_answer_str = data.load_validation()

train_doc, train_answer = tk.tokenize_set(train_doc_str, train_answer_str, tokenizer)
test_doc, test_answer = tk.tokenize_set(test_doc_str, test_answer_str, tokenizer)
val_doc, val_answer = tk.tokenize_set(val_doc_str, val_answer_str, tokenizer)

# Sanity check
# logging.info("Sanity check: %s",metrics.precision(test_answer,test_answer))

logging.info("Dataset loaded. Preprocessing data...")

train_x, train_y, test_x, test_y, val_x, val_y, embedding_matrix = preprocessing. \
    prepare_sequential(train_doc, train_answer, test_doc, test_answer, val_doc, val_answer,
                       max_document_length=MAX_DOCUMENT_LENGTH,
                       max_vocabulary_size=MAX_VOCABULARY_SIZE,
                       embeddings_size=EMBEDDINGS_SIZE,
                       stem_test=STEM_TEST)

# weigh training examples: everything that's not class 0 (not kp)
# gets a heavier score
from sklearn.utils import class_weight

train_y_weights = np.argmax(train_y, axis=2)
train_y_weights = np.reshape(class_weight.compute_sample_weight('balanced', train_y_weights.flatten()),
                             np.shape(train_y_weights))

logging.info("Data preprocessing complete.")
logging.info("Maximum possible recall: %s",
             metrics.recall(test_answer,
                            postprocessing.get_words(test_doc, postprocessing.undo_sequential(test_y)),
                            STEM_MODE))

if not SAVE_MODEL or not os.path.isfile(MODEL_PATH):

    logging.debug("Building the network...")
    model = Sequential()

    embedding_layer = Embedding(np.shape(embedding_matrix)[0],
                                EMBEDDINGS_SIZE,
                                weights=[embedding_matrix],
                                input_length=MAX_DOCUMENT_LENGTH,
                                trainable=False)

    model.add(embedding_layer)
    model.add(Bidirectional(LSTM(300, activation='tanh', recurrent_activation='hard_sigmoid', return_sequences=True)))
    model.add(Dropout(0.25))
    model.add(TimeDistributed(Dense(150, activation='relu', kernel_regularizer=regularizers.l2(0.01))))
    model.add(Dropout(0.25))
    model.add(TimeDistributed(Dense(3, activation='softmax')))

    logging.info("Compiling the network...")
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'],
                  sample_weight_mode="temporal")
    print(model.summary())

    metrics_callback = keras_metrics.MetricsCallback(val_x, val_y)

    logging.info("Fitting the network...")

    history = model.fit(train_x, train_y,
                        validation_data=(val_x, val_y),
                        epochs=EPOCHS,
                        batch_size=BATCH_SIZE,
                        sample_weight=train_y_weights,
                        callbacks=[metrics_callback])

    if SHOW_PLOTS:
        plots.plot_accuracy(history)
        plots.plot_loss(history)
        plots.plot_prf(metrics_callback)

    if SAVE_MODEL:
        model.save(MODEL_PATH)
        logging.info("Model saved in %s", MODEL_PATH)

else:
    logging.info("Loading existing model from %s...", MODEL_PATH)
    model = load_model(MODEL_PATH)
    logging.info("Completed loading model from file")

logging.info("Predicting on test set...")
output = model.predict(x=test_x, verbose=1)
logging.debug("Shape of output array: %s", np.shape(output))

obtained_tokens = postprocessing.undo_sequential(output)
obtained_words = postprocessing.get_words(test_doc, obtained_tokens)

precision = metrics.precision(test_answer, obtained_words,STEM_MODE)
recall = metrics.recall(test_answer, obtained_words,STEM_MODE)
f1 = metrics.f1(precision, recall)

print("###    Obtained Scores    ###")
print("###     (full dataset)    ###")
print("###")
print("### Precision : %.4f" % precision)
print("### Recall    : %.4f" % recall)
print("### F1        : %.4f" % f1)
print("###                       ###")

keras_precision = keras_metrics.keras_precision(test_y, output)
keras_recall = keras_metrics.keras_recall(test_y, output)
keras_f1 = keras_metrics.keras_f1(test_y, output)

print("###    Obtained Scores    ###")
print("###    (fixed dataset)    ###")
print("###")
print("### Precision : %.4f" % keras_precision)
print("### Recall    : %.4f" % keras_recall)
print("### F1        : %.4f" % keras_f1)
print("###                       ###")

clean_words = postprocessing.get_valid_patterns(obtained_words)

precision = metrics.precision(test_answer, clean_words,STEM_MODE)
recall = metrics.recall(test_answer, clean_words,STEM_MODE)
f1 = metrics.f1(precision, recall)

print("###    Obtained Scores    ###")
print("### (full dataset,        ###")
print("###  pos patterns filter) ###")
print("###")
print("### Precision : %.4f" % precision)
print("### Recall    : %.4f" % recall)
print("### F1        : %.4f" % f1)
print("###                       ###")

obtained_words_top = postprocessing.get_top_words(test_doc, output, 5)

precision_top = metrics.precision(test_answer, obtained_words_top,STEM_MODE)
recall_top = metrics.recall(test_answer, obtained_words_top,STEM_MODE)
f1_top = metrics.f1(precision_top, recall_top)

print("###    Obtained Scores    ###")
print("### (full dataset, top 5) ###")
print("###")
print("### Precision : %.4f" % precision_top)
print("### Recall    : %.4f" % recall_top)
print("### F1        : %.4f" % f1_top)
print("###                       ###")

obtained_words_top = postprocessing.get_top_words(test_doc, output, 10)

precision_top = metrics.precision(test_answer, obtained_words_top,STEM_MODE)
recall_top = metrics.recall(test_answer, obtained_words_top,STEM_MODE)
f1_top = metrics.f1(precision_top, recall_top)

print("###    Obtained Scores    ###")
print("### (full dataset, top 10)###")
print("###")
print("### Precision : %.4f" % precision_top)
print("### Recall    : %.4f" % recall_top)
print("### F1        : %.4f" % f1_top)
print("###                       ###")

obtained_words_top = postprocessing.get_top_words(test_doc, output, 15)

precision_top = metrics.precision(test_answer, obtained_words_top,STEM_MODE)
recall_top = metrics.recall(test_answer, obtained_words_top,STEM_MODE)
f1_top = metrics.f1(precision_top, recall_top)

print("###    Obtained Scores    ###")
print("### (full dataset, top 15)###")
print("###")
print("### Precision : %.4f" % precision_top)
print("### Recall    : %.4f" % recall_top)
print("### F1        : %.4f" % f1_top)
print("###                       ###")

if DATASET == Semeval2017:
    print("### SEMEVAL ANNOTATOR ###")
    print("###        All        ###")

    from eval import anno_generator

    anno_generator.write_anno("/tmp/simplernn", test_doc_str, obtained_words)
    from data.Semeval2017 import eval

    eval.calculateMeasures("data/Semeval2017/test", "/tmp/simplernn-all", remove_anno=["types"])

    print("###     Filtered      ###")
    anno_generator.write_anno("/tmp/simplernn-clean", test_doc_str, clean_words)
    eval.calculateMeasures("data/Semeval2017/test", "/tmp/simplernn-clean", remove_anno=["types"])

    print("###      Top 15       ###")
    anno_generator.write_anno("/tmp/simplernn-15", test_doc_str, obtained_words_top)
    eval.calculateMeasures("data/Semeval2017/test", "/tmp/simplernn-15", remove_anno=["types"])

    print("###  Top 15 Filtered  ###")
    clean_words = postprocessing.get_valid_patterns(obtained_words_top)

    anno_generator.write_anno("/tmp/simplernn-15-clean", test_doc_str, clean_words)
    eval.calculateMeasures("data/Semeval2017/test", "/tmp/simplernn-15-clean", remove_anno=["types"])

2019-12-30 21:37:08,669	INFO	Keras version 2.2.5
2019-12-30 21:37:08,670	INFO	Numpy version 1.17.4
2019-12-30 21:37:08,670	INFO	Tensorflow version 1.15.0
2019-12-30 21:37:08,672	INFO	Loading dataset...
2019-12-30 21:37:08,673	DEBUG	Initialized dataset Bangla from folder data/Bangla
2019-12-30 21:37:08,790	DEBUG	Loaded training set for dataset Bangla
2019-12-30 21:37:08,896	DEBUG	Loaded test set for dataset Bangla
2019-12-30 21:37:08,913	DEBUG	Loaded validation set for dataset Bangla
2019-12-30 21:37:18,601	INFO	Dataset loaded. Preprocessing data...
2019-12-30 21:37:20,534	DEBUG	Fitting dictionary on 3208 documents...
2019-12-30 21:37:21,041	DEBUG	Dictionary fitting completed. Found 58394 unique tokens
2019-12-30 21:37:21,468	DEBUG	Longest training document : 1888 tokens
2019-12-30 21:37:21,469	DEBUG	Longest test document :     1888 tokens
2019-12-30 21:37:21,470	DEBUG	Longest validation document : 1550 tokens
2019-12-30 21:37:21,967	DEBUG	Training set samples size   : (1500, 550)
2019-

2019-12-30 21:37:47,475	WARNING	From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2019-12-30 21:37:47,485	WARNING	From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2019-12-30 21:37:47,491	WARNING	From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



2019-12-30 21:37:47,500	WARNING	From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



...

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 550, 300)          6000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 550, 600)          1442400   
_________________________________________________________________
dropout_1 (Dropout)          (None, 550, 600)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 550, 150)          90150     
_________________________________________________________________
dropout_2 (Dropout)          (None, 550, 150)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 550, 3)            453       
Total params: 7,533,003
Trainable params: 1,533,003
Non-trainable params: 6,000,000
____________________________________

2019-12-30 21:37:55,881	WARNING	From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2019-12-30 21:37:56,881	WARNING	From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



2019-12-30 21:37:56,889	WARNING	From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 1500 samples, validate on 208 samples
Epoch 1/28
1500/1500 [==============================] - 73s 48ms/step - loss: 2.0375 - acc: 0.5743 - val_loss: 1.2244 - val_acc: 0.9106

###   Validation Scores   ###
###
### Epoch     : 1
### Precision : 0.0895
### Recall    : 0.0584
### F1        : 0.0707
###                       ###
Epoch 2/28
1500/1500 [==============================] - 71s 48ms/step - loss: 1.2212 - acc: 0.3268 - val_loss: 1.1437 - val_acc: 0.0383

###   Validation Scores   ###
###
### Epoch     : 2
### Precision : 0.0295
### Recall    : 0.8698
### F1        : 0.0570
###                       ###
Epoch 3/28
1500/1500 [==============================] - 70s 47ms/step - loss: 1.1082 - acc: 0.4002 - val_loss: 1.0608 - val_acc: 0.7806

###   Validation Scores   ###
###
### Epoch     : 3
### Precision : 0.0777
### Recall    : 0.0262
### F1        : 0.0392
###                       ###
Epoch 4/28
1500/1500 [==============================] - 70s 47ms/step - loss: 1.1001 - ac